# HW1

In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]
grades_list = [1, 2, 3, 1, 1, 2, 3, 1, 2, 1]
prices_recommended = [400, 60, 40, 40 , 90]
prices_bought = [60, 40, 40 , 90]

## Hit Rate

In [3]:
def hit_rate(recommended_list, bought_list):

    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)

    flags = np.isin(bought_list, recommended_list)

    hit_rate = (flags.sum() > 0) * 1

    return hit_rate


def hit_rate_at_k(recommended_list, bought_list, k=5):

    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k]) # Срез до k в листе рекоммендаций.

    flags = np.isin(bought_list, recommended_list)

    hit_rate = (flags.sum() > 0) * 1

    return hit_rate

In [4]:
hit_rate_at_k(recommended_list, bought_list, k=5)

1

## Precision

In [5]:
def precision(recommended_list, bought_list):

    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)

    flags = np.isin(bought_list, recommended_list)

    precision = flags.sum() / len(recommended_list)

    return precision


def precision_at_k(recommended_list, bought_list, k=5):

    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)

    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]

    flags = np.isin(bought_list, recommended_list)

    precision = flags.sum() / len(recommended_list)


    return precision



def money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5):

    recommend_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]

    flags = np.isin(recommend_list, bought_list)

    precision = np.dot(flags, prices_recommended) / prices_recommended.sum()

    return precision

In [6]:
money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5)

0.6984126984126984

## Recall


In [7]:
def recall(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)

    flags = np.isin(bought_list, recommended_list)

    recall = flags.sum() / len(bought_list)

    return recall


def recall_at_k(recommended_list, bought_list, k=5):
    # your_code

    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)

    recommended_list = recommended_list[:k]

    flags = np.isin(bought_list, recommended_list)

    recall = flags.sum() / len(bought_list)

    return recall


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    # your_code

    bought_list = np.array(bought_list)
    prices_bought = np.array(prices_bought)
    recommended_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]

    flags = np.isin(recommended_list, bought_list)

    recall = np.dot(flags, prices_recommended) / prices_bought.sum()

    return recall

In [8]:
recall_at_k(recommended_list, bought_list, 5)

0.5

In [9]:
money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, 5)

1.9130434782608696

## MAP@k

In [10]:
def ap_k(recommended_list, bought_list, k=5):

    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)

    flags = np.isin(recommended_list, bought_list)

    if sum(flags) == 0:
        return 0

    sum_ = 0
    for i in range(0, k-1):
        if flags[i] == True:
            p_k = precision_at_k(recommended_list, bought_list, k=i+1)
            sum_ += p_k

    result = sum_ / sum(flags)

    return result

def map_k(recommended_all_users, bought_all_users, k=5):

    # your_code
    sum_ap_k = sum([ap_k(recommended_all_users[u], bought_all_users[u], k) for u in range(len(bought_all_users))])

    map_k = sum_ap_k / len(bought_all_users)

    return map_k

In [11]:
# Требуются данные рекоммендаций и покупок для разных пользователей, сгенерирую на лету.
map_k([recommended_list for i in range(5)], [bought_list[:i] for i in range(5)], k=5)

0.35

## AP для примера с лекции

In [12]:
_recommended_list = [6, 3, 5, 0, 4, 2, 1, 7]
_bought_list = [6, 5, 0, 2]


In [13]:
ap_k(_recommended_list, _bought_list, k=5)

0.6041666666666666

In [14]:
for i in range(1, 9):
    print(i, ap_k(_recommended_list, _bought_list, k=i))

1 0.0
2 0.25
3 0.25
4 0.41666666666666663
5 0.6041666666666666
6 0.6041666666666666
7 0.7708333333333333
8 0.7708333333333333


## MRR@K

In [15]:
def reciprocal_rank(recommended_all_users, bought_all_users, k=5):
    recommended_all_users = np.array(recommended_all_users)
    bought_all_users = np.array(bought_all_users)

    rr_list = []
    for i in np.arange(len(bought_list)):
        relevant_indexes = np.nonzero(np.isin(recommended_all_users[i][:k], bought_all_users[i]))[0]
        if len(relevant_indexes) != 0:
            rr_list.append(1/(relevant_indexes[0]+1))


    return sum(rr_list)/len(bought_list)

In [16]:
# Требуются данные рекоммендаций и покупок для разных пользователей, сгенерирую на лету.
reciprocal_rank([recommended_list for i in range(5)], [bought_list[:i] for i in range(5)], k=5)

/home/pavel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  This is separate from the ipykernel package so we can avoid doing imports until


0.25

## NDCG@K

In [18]:
grades_list = [3, 2, 1, 1, 3, 1, 2]

In [19]:
def dcg_at_k(grades_list, k=7):

    grades_list = np.array(grades_list)[:k]

    dcg = 0
    for i in range(len(grades_list)):
        if i == 0:
            dcg = grades_list[i]
            # print(grades_list[i])
        else:
            dcg += (grades_list[i] / math.log2(i+1+1))
            # print(grades_list[i], ' / ', math.log2(i+2))

    return dcg

In [20]:
dcg_at_k(grades_list, k=7)

7.375968340694622

In [21]:
dcg_at_k(sorted(grades_list, reverse=True), k=7)

7.830535704537055

In [22]:
def ndcg_at_k(grades_list, k=7):


    dcg = dcg_at_k(grades_list, k)
    ideal_dcg = dcg_at_k(sorted(grades_list, reverse=True), k)

    ndcg = dcg / ideal_dcg

    return ndcg


In [23]:
ndcg_at_k(grades_list, k=7)

0.9419493913323128